In [1]:
import h5py as hdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nexusformat.nexus as nx
from scipy.interpolate import griddata
from lib.reader import *
from lib.helpers import *
from lib.imports import *
from lib.filters import *
from lib.visualize import *
plt.style.use('M:\\Betrieb_ISF\\Gruppe_A_Orga\\Arbeitsgebiete\\Python_Auswerteskripte\\example_file.mplstyle')


In [2]:
path='C:\\Users\\maede\\DESY\\data\\laht\\d5kW_oZW_LAHT-044.nxs'
f= nx.nxload(path)
print (f.tree)
files=gather_files_regex('C:\\Users\\maede\\DESY\\data\\laht',r'\S*nxs\Z')

NeXusError: 'C:\Users\maede\DESY\data\laht/' is not accessible

In [3]:

df0,df1=load_nxs(files[2])
dfr0=get_energy_peaks(df0)
dfr0=separate_energy_peak(dfr0,110,120)
disp_peak_move(dfr0)

NameError: name 'files' is not defined

In [6]:
exs=pd.read_excel('experiments.xlsx',sheet_name='Tabelle3',header=0)
exs['data']=exs.Messdatei.apply(load_nxs)

NeXusError: 'G:\2021-12-07_Desy_Messdaten\complete\raw\HA_001\experiments/' is not accessible

In [ ]:
exs['ch0']=exs['data'].apply(lambda x :x[0] )
exs['ch1']=exs['data'].apply(lambda x :x[1] )

exs['ch0_energypeaks']=exs['ch0'].apply(get_energy_peaks)
exs['ch1_energypeaks']=exs['ch1'].apply(get_energy_peaks)

exs['ch0_energypeaks']=exs['ch0_energypeaks'].apply(lambda x : separate_energy_peak(x,103,110))
exs['ch1_energypeaks']=exs['ch1_energypeaks'].apply(lambda x : separate_energy_peak(x,103,110))

exs['ch0_ss']= exs['ch0_energypeaks'].apply(extract_ss_mean)
exs['ch1_ss']= exs['ch1_energypeaks'].apply(extract_ss_mean)



In [ ]:
groups={'HA':1,
        'HA3Si1':2,
        'LAHT':3,
        'HAG199':4,
        'LAHTG199':5,
        'LAG199':6,
        'LA3Si1':7,
        'HAref':11,
        'HAucref':12,
        'LAHTref':31,
        'LAHTucref':32,
        'LAref':61,
        }
groups

In [ ]:
cm=2.5
dfs=exs[exs.Gruppe==groups['LAHT']]
x=-dfs.X
y=-dfs.Z
z=dfs['ch1_ss']
z=z.fillna(exs[exs.Gruppe==groups['LAHTref']]['ch0_ss'].mean())
#z=z.fillna(105.90972527472528)
105.90972527472528
xedges = np.arange(*minmax(x))
yedges = np.arange(*minmax(y))

xcenters = xedges[:-1] + np.diff(xedges[:2])[0] / 2.
ycenters = yedges[:-1] + np.diff(yedges[:2])[0] / 2.

XI, YI = np.meshgrid(xcenters, ycenters)
grid_z0 = griddata(list(zip(x,y)), z, (XI, YI), method='nearest')

fig,ax=plt.subplots(figsize=(20/cm,4/cm))
a=ax.imshow(sgolay2d(grid_z0,9,3),  origin='lower',cmap='plasma')
#a=ax.imshow(grid_z0,  origin='lower',cmap='plasma')

ax.scatter(x+10, y+20, 20,z,  edgecolor='k',lw=0.5,  cmap='plasma')
ax.set_xticklabels([])
ax.set_yticklabels([])
cbar=fig.colorbar(a)
cbar.ax.tick_params(size=0)
cbar.set_ticks([])
#cbar.set_ticks([a.colorbar.vmin + t*(a.colorbar.vmax-a.colorbar.vmin) for t in cbar.ax.get_yticks()])
print(cbar.ax.get_yticks())
#plt.axis('equal')
plt.show()
fig.savefig('result')

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
for i,val in enumerate(dfs['ch1_energypeaks']):
    ax.plot(val.iloc[:,1],'o',markersize=1,label=i)
plt.legend()
plt.show()

In [ ]:
display(dfs)